In [1]:
import numpy as np
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -qq langdetect
from langdetect import detect
import string
!pip install -qq contractions
!pip install -qq demoji
import demoji
demoji.download_codes()
!pip install -qq datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.4 MB/s eta 0:00:00


<ipython-input-2-28a5514cb77a>:7: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00


In [3]:
header_names = ['label', 'text']
df_train = pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/mal_full_offensive.csv", sep='\t',names=header_names)
df_train = df_train.dropna(how='all')
df_train = pd.DataFrame(df_train)
df_train

,label,text
0,Not_offensive,Apt മ്യൂസിക് Sushin.. Hats off
1,Not_offensive,മിഥുൻ ചേട്ടാ ഇങ്ങനത്തെ ഐറ്റം കയ്യിൽ വെച്ചിട്ട...
2,not-malayalam,Too good ....shyju khilad again ... wonderful ...
3,Not_offensive,രാക്ഷസൻ ഇമൈകൾനോടികൾ ഒക്കെ അങ്ങോട്ട് മാറിനിക്ക്...
4,Not_offensive,മലയാളത്തിനും ഉണ്ടെന്ന് പറഞ്ഞേക് അസ്സൽ ത്രില്ലർ...
...,...,...
20005,Not_offensive,വില്ലൻ -ശ്രീനാഥ് ഭാസി
20006,Not_offensive,തുടക്കത്തിലെ മൂസിക് കേട്ടപ്പോ തന്നെ വീഡിയോ സ്റ...
20007,Not_offensive,കുഞ്ചാക്കോ ബോബൻ ഇങ്ങനത്തെ റോളുകൾ ചെയ്യുമ്പോൾ.....
20008,Not_offensive,Nalla parichayam... Aetho Korean movie manakku...


In [4]:
label_mapping = {
    'Not_offensive':0,
    'Offensive_Untargetede':1,
    'Offensive_Targeted_Insult_Individual':1,
    'Offensive_Targeted_Insult_Group':1,
    'Offensive_Targeted_Insult_Other':1,
    'not-malayalam':2,
}
df_train['label'].replace(label_mapping, inplace=True)
df_train = df_train[df_train['label'] != 2]
print(df_train.info())
label_counts = df_train['label'].value_counts()
print(label_counts)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18403 entries, 0 to 20009
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   18403 non-null  int64 
 1   text    18403 non-null  object
dtypes: int64(1), object(1)
memory usage: 431.3+ KB
None
0    17697
1      706
Name: label, dtype: int64


In [5]:
df_train.head(20)

,label,text
0,0,Apt മ്യൂസിക് Sushin.. Hats off
1,0,മിഥുൻ ചേട്ടാ ഇങ്ങനത്തെ ഐറ്റം കയ്യിൽ വെച്ചിട്ട...
3,0,രാക്ഷസൻ ഇമൈകൾനോടികൾ ഒക്കെ അങ്ങോട്ട് മാറിനിക്ക്...
4,0,മലയാളത്തിനും ഉണ്ടെന്ന് പറഞ്ഞേക് അസ്സൽ ത്രില്ലർ...
5,0,മലയാളത്തിലെ രാക്ഷസൻ ആവാനുള്ള എല്ലാ ലക്ഷണവുമുണ്...
6,0,ഇത് പോലെ ഞെട്ടിച്ച ട്രൈലർ വേറൊന്നും അടുത്ത് കണ...
7,0,വേട്ടക്ക് ശേഷം ചാക്കോച്ചന്റെ കിടിലൻ ത്രില്ലെർ ...
8,0,പടം പൊട്ടില്ല എന്ന് ഉറപ്പുള്ളവർക് like അടിക്കാ...
9,0,ഷറഫുദ്ധീൻ ആണെന്ന് തോന്നുന്നു ഈ സീരിയൽ കില്ലർ
10,0,Ente ponno Malayalam rakshasan loading


In [6]:
import sys
sys.path.append("/content/drive/MyDrive/Capstone/Translator/")
from google_trans_new import google_translator

In [8]:
translator = google_translator()
for index, row in df_train.iterrows(): # Check if emoji is contained in text
    text = str(row['text'])
    found = demoji.findall(text)
    if found:
        # There is an emoji now detect language
        if detect(text) == 'ml': # Text is Malayalam
            for key, value in found.items():
                emojiEng = value
                # Translate to Malayalam
                transText = translator.translate(emojiEng, lang_tgt='ml', lang_src='en')
                text = text.replace(key, transText + ' ')
        else: # Text is english
            for key, value in found.items():
                text = text.replace(key, value + ' ')

        df_train.at[index, 'text'] = text

In [9]:
df_train.head(30)

,label,text
0,0,Apt മ്യൂസിക് Sushin.. Hats off
1,0,മിഥുൻ ചേട്ടാ ഇങ്ങനത്തെ ഐറ്റം കയ്യിൽ വെച്ചിട്ട...
3,0,രാക്ഷസൻ ഇമൈകൾനോടികൾ ഒക്കെ അങ്ങോട്ട് മാറിനിക്ക്...
4,0,മലയാളത്തിനും ഉണ്ടെന്ന് പറഞ്ഞേക് അസ്സൽ ത്രില്ലർ...
5,0,മലയാളത്തിലെ രാക്ഷസൻ ആവാനുള്ള എല്ലാ ലക്ഷണവുമുണ്...
6,0,ഇത് പോലെ ഞെട്ടിച്ച ട്രൈലർ വേറൊന്നും അടുത്ത് കണ...
7,0,വേട്ടക്ക് ശേഷം ചാക്കോച്ചന്റെ കിടിലൻ ത്രില്ലെർ ...
8,0,പടം പൊട്ടില്ല എന്ന് ഉറപ്പുള്ളവർക് like അടിക്കാ...
9,0,ഷറഫുദ്ധീൻ ആണെന്ന് തോന്നുന്നു ഈ സീരിയൽ കില്ലർ
10,0,Ente ponno Malayalam rakshasan loading


In [10]:
for index, row in df_train.iterrows():
    found = demoji.findall(row['text'])
    if found:
        print('Found exception')

In [11]:
import contractions
# contracted text
for index, row in df_train.iterrows():# creating an empty list
    expanded_words = []
    for word in str(row['text']).split():# using contractions.fix to expand the shotened words
        expanded_words.append(contractions.fix(word))

    expanded_text = ' '.join(expanded_words)
    df_train.at[index, 'text'] = expanded_text


for index, row in df_train.iterrows():
    Sentence = str(row['text'])
    for i in Sentence:
        # checking whether the char is punctuation.
        if i in string.punctuation:
            Sentence = Sentence.replace(i, '')
    df_train.at[index, 'text'] = Sentence.lower()

In [12]:
df_train.head(30)

,label,text
0,0,apt മ്യൂസിക് sushin hats off
1,0,മിഥുൻ ചേട്ടാ ഇങ്ങനത്തെ ഐറ്റം കയ്യിൽ വെച്ചിട്ടാ...
3,0,രാക്ഷസൻ ഇമൈകൾനോടികൾ ഒക്കെ അങ്ങോട്ട് മാറിനിക്ക്...
4,0,മലയാളത്തിനും ഉണ്ടെന്ന് പറഞ്ഞേക് അസ്സൽ ത്രില്ലർ...
5,0,മലയാളത്തിലെ രാക്ഷസൻ ആവാനുള്ള എല്ലാ ലക്ഷണവുമുണ്ട്
6,0,ഇത് പോലെ ഞെട്ടിച്ച ട്രൈലർ വേറൊന്നും അടുത്ത് കണ...
7,0,വേട്ടക്ക് ശേഷം ചാക്കോച്ചന്റെ കിടിലൻ ത്രില്ലെർ ...
8,0,പടം പൊട്ടില്ല എന്ന് ഉറപ്പുള്ളവർക് like അടിക്കാ...
9,0,ഷറഫുദ്ധീൻ ആണെന്ന് തോന്നുന്നു ഈ സീരിയൽ കില്ലർ
10,0,ente ponno malayalam rakshasan loading


In [13]:
malayalam_stopwords = [
    "അത്", "ഇത്", "അവന്", "അവള്", "എന്റെ",
    "എന്നേക്കും", "എന്നേക്കും", "എന്നു", "എന്നും", "എന്ന്",
    "എന്നുമുണ്ട്", "എന്നുമ്", "എന്നുമ്", "എന്നും", "എന്നൈക്കൊണ്ട്",
    "എന്നും", "എന്നുമ്", "എന്നേക്കൊണ്ട്", "ഏത്", "ഒരു",
    "ഒരുവർ", "ഒരുവളുകൾ", "എന്ന", "എന്നത്",
    "എന്നതായി", "എന്നാണ്", "എന്നാൽ", "എന്നോ", "എന്നോട്",
    "എന്ന്", "ഇനി", "ഇപ്പോൾ", "ഇതിൽ", "അല്ല",
    "ഇതുമുതൽ", "ഇതിൽ", "എന്നുകൊണ്ട്", "ഇതുപോലെ", "അതു",
    "ഇതിനെ", "ഇതാ", "അതാ", "അതായത്",
    "ഇതായത്", "ഇതിന്റെ", "അതിന്റെ", "ഇതിൽ", "ഇന്ന്",
    "ഇന്നത്തെ", "ഇന്നു", "ഇന്നുകഴിഞ്ഞ്", "ഇന്നുകൊണ്ട്", "ഇന്നുവരെ",
    "ഇന്നുവർ"
]


In [14]:
data = []

for index, row in df_train.iterrows():
    sent = ''
    line = str(row['text'])
    for i in line.split():
        if i not in malayalam_stopwords:
            sent += i + " "
    data.append({'text': sent, 'label': str(row['label'])})

new_df_train = pd.DataFrame(data)
new_df_train.head(30)


,text,label
0,apt മ്യൂസിക് sushin hats off,0
1,മിഥുൻ ചേട്ടാ ഇങ്ങനത്തെ ഐറ്റം കയ്യിൽ വെച്ചിട്ടാ...,0
2,രാക്ഷസൻ ഇമൈകൾനോടികൾ ഒക്കെ അങ്ങോട്ട് മാറിനിക്ക്...,0
3,മലയാളത്തിനും ഉണ്ടെന്ന് പറഞ്ഞേക് അസ്സൽ ത്രില്ലർ...,0
4,മലയാളത്തിലെ രാക്ഷസൻ ആവാനുള്ള എല്ലാ ലക്ഷണവുമുണ്ട്,0
5,പോലെ ഞെട്ടിച്ച ട്രൈലർ വേറൊന്നും അടുത്ത് കണ്ടിട...,0
6,വേട്ടക്ക് ശേഷം ചാക്കോച്ചന്റെ കിടിലൻ ത്രില്ലെർ ...,0
7,പടം പൊട്ടില്ല ഉറപ്പുള്ളവർക് like അടിക്കാം നല്ല...,0
8,ഷറഫുദ്ധീൻ ആണെന്ന് തോന്നുന്നു ഈ സീരിയൽ കില്ലർ,0
9,ente ponno malayalam rakshasan loading,0


In [15]:
new_df_train['text'].isnull().sum()

0

In [16]:
!pip install -qq indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 13.5 MB/s eta 0:00:00


In [17]:
from indicnlp import common
from indicnlp import loader
from indicnlp.script import indic_scripts
from indicnlp.tokenize import indic_tokenize
from indicnlp.morph import unsupervised_morph

In [18]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 20.81 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (28/28), done.


In [19]:
INDIC_NLP_RESOURCES = r"/content/indic_nlp_resources"
common.set_resources_path(INDIC_NLP_RESOURCES)
loader.load()
analyzer = unsupervised_morph.UnsupervisedMorphAnalyzer('ml')

In [20]:
from langdetect import detect, LangDetectException

In [21]:
from tqdm import tqdm

def stem_malayalam_text(text):
    tokens = text.split()
    stemmed_tokens = []
    for token in tokens:
        analyzed_tokens = analyzer.morph_analyze_document([token])
        if analyzed_tokens:
            stemmed_tokens.append(analyzed_tokens[0])
        else:
            stemmed_tokens.append(token)  # If stemming fails, keep the original token
    return ' '.join(stemmed_tokens)

# Assuming new_df_train is your DataFrame containing text data
# Add tqdm to the loop to track progress
for index, row in tqdm(new_df_train.iterrows(), total=len(new_df_train)):
    text = str(row['text'])

    try:
        detected_lang = detect(text)
    except LangDetectException as e:
        # Handle the exception, for example by setting the detected_lang to None
        detected_lang = None

    if detected_lang == 'ml':
        stemmed_text = stem_malayalam_text(text)
        new_df_train.at[index, 'text'] = stemmed_text


print(new_df_train.head(20))


100%|██████████| 18403/18403 [03:48<00:00, 80.46it/s] 

                                                 text label
0                       apt മ്യൂസിക് sushin hats off      0
1   മിഥു ചേട്ട ഇങ്ങ ഐറ്റം കയ് വെച്ച കോമഡി ഫിലിം ചെ...     0
2           രാക്ഷസ ഇ ഒക്കെ അ മാറി ഇവിടെ 5ആം പാ ഭരിക്ക     0
3            മലയാള ഉണ്ടെന്ന പറഞ്ഞ അസ്സ ത്രില് waiting     0
4                          മലയാള രാക്ഷസ ആ എല്ലാ ലക്ഷണ     0
5            പോലെ ഞെട്ടിച്ച ട്രൈ വേറ അടുത്ത കണ്ട സൂപ്     0
6       വേട്ട ശേഷം ചാക്കോച്ചന് കിട ത്രില് ആ തോന്നുന്ന     0
7         പടം പൊട്ട ഉറപ്പുള്ള like അടി നല്ല ത്രില് ആയ     0
8                   ഷറ ആണെന്ന തോന്നുന്ന ഈ സീരിയ കില്ല     0
9             ente ponno malayalam rakshasan loading      0
10               വേട്ട ശേഷം അടുത്ത ത്രില് iam waiting     0
11  ശ്രീ ഭാസി sb ഫാൻസ്‌ ലൈക്‌ അടി സ്നേഹ ആംഗ്യം സ്ന...     0
12                            മിഥു comedy ഇല്ലാതെ പടം     0
13                          ചാക്കോ ച്ച ഒന്നൊന് വരവ മോ     0
14  മിഥു എല്ല സിനിമ ചെയ്യുന്ന മികച്ച സംവിധായക ആണ് ...     0
15  ചാക്കോച്ചന് ല ഫാൻ വക എല്ലാ ആശംസ അഞ്ച

In [22]:
new_df_train.to_csv('malayalam_dataset_2_train.csv', index=False)